# IMDB Investigation

### Import Library

In [1]:
# import
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook


### Import Data

In [2]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

basics = pd.read_csv(basics_url, low_memory=False, sep='\t')
akas = pd.read_csv(akas_url,  low_memory=False, sep='\t')
ratings = pd.read_csv(ratings_url, low_memory=False, sep='\t')

### Review Dataframes

In [3]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
#creating backup of 'basics' dataframe
df_b_save = basics.copy()

In [5]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [6]:
df_a_save = akas.copy()

In [7]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
2,tt0000003,6.5,1836
3,tt0000004,5.6,179
4,tt0000005,6.2,2625


In [8]:
df_r_save = ratings.copy()

### Preprocessing Cleanup

#### title.basics

##### Find NAN Values

In [9]:
basics = basics.replace({'\\N':np.nan})

##### Evaluating NAN values in columns

In [10]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1339594
endYear           9831148
runtimeMinutes    6996180
genres             445227
dtype: int64

In [11]:
#Display the percentage of missing values by row. 
print(basics.isna().sum()/len(basics)*100)

tconst             0.000000
titleType          0.000000
primaryTitle       0.000111
originalTitle      0.000111
isAdult            0.000010
startYear         13.476866
endYear           98.905387
runtimeMinutes    70.384445
genres             4.479166
dtype: float64


##### Dropping column endYear since this primarily an investigation of movies

In [12]:
basics.drop(['endYear'], axis=1, inplace=True)

##### Eliminate movies that are null for runtimeMinutes

In [13]:
basics.dropna(subset=['runtimeMinutes'], inplace=True)

##### Eliminate movies that are null for genre

In [14]:
basics.dropna(subset=['genres'], inplace=True)

##### keep only titleType==movie

In [15]:
basics = basics[basics['titleType'] == 'movie']

##### Eliminating NAN values for 'startYear'

In [16]:
basics.dropna(subset=['startYear'], inplace=True)

##### NAN values resolved

In [17]:
print(basics.isna().sum()/len(basics)*100)

tconst            0.0
titleType         0.0
primaryTitle      0.0
originalTitle     0.0
isAdult           0.0
startYear         0.0
runtimeMinutes    0.0
genres            0.0
dtype: float64


##### Modifying Datatypes to conform to data dictionary

In [18]:
basics = basics.astype({"tconst":"string","titleType":"string","primaryTitle":"string","originalTitle":"string", "isAdult":"int", "startYear":"int","runtimeMinutes":"int", "genres":"string"})

In [19]:
basics.dtypes

tconst            string
titleType         string
primaryTitle      string
originalTitle     string
isAdult            int32
startYear          int32
runtimeMinutes     int32
genres            string
dtype: object

##### keep startYear 2000-2022

In [20]:
basics = basics[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2022)]

##### Eliminate movies that include "Documentary" in genre

In [21]:
basics = basics[~basics['genres'].str.contains(r'documentary', case=False)]


#### title.akas

##### Find NAN Values

In [22]:
akas =akas.replace({'\\N':np.nan})

##### Filtering out all non US Films

In [23]:
akas = akas[akas['region'] == 'US']

##### Evaluating NAN values in columns

In [24]:
akas.isna().sum()

titleId                  0
ordering                 0
title                    0
region                   0
language           1441942
types               465613
attributes         1399096
isOriginalTitle       1342
dtype: int64

In [25]:
print(akas.isna().sum()/len(akas)*100)

titleId             0.000000
ordering            0.000000
title               0.000000
region              0.000000
language           99.726605
types              32.202408
attributes         96.763320
isOriginalTitle     0.092814
dtype: float64


##### Dropping Columns Language and Attributes due to lack of data, as well as isOriginalTitle since it doesnt directly relate to investigation

In [26]:
akas.drop(['language','attributes','isOriginalTitle'], axis=1, inplace=True)

##### Replacing NAN values with Unknown for types column

In [27]:
type_of = 'unknown'
akas['types'].fillna(value = type_of, inplace = True)

##### NAN Values Resolved

In [28]:
akas.isna().sum()

titleId     0
ordering    0
title       0
region      0
types       0
dtype: int64

##### Modifying Datatypes to conform to Data Dictionary

In [29]:
akas = akas.astype({"titleId":"string","title":"string","region":"string","types":"string"})

In [30]:
akas.dtypes

titleId     string
ordering     int64
title       string
region      string
types       string
dtype: object

#### title.ratings

##### Find NAN Values

In [31]:
ratings = ratings.replace({'\\N':np.nan})

In [32]:
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

* No NAN values found

##### Modifying Datatypes to conform to data dictionary

In [33]:
ratings = ratings.astype({"tconst":"string"})

In [34]:
ratings.dtypes

tconst            string
averageRating    float64
numVotes           int64
dtype: object

#### Filtering for Title Basics with akas dataframe for US only

In [35]:
keepers1 =basics['tconst'].isin(akas['titleId'])
keepers1

34803       True
42384       True
61115       True
67667       True
86799       True
           ...  
9939625     True
9939634     True
9939673    False
9939718     True
9939802    False
Name: tconst, Length: 147738, dtype: bool

In [36]:
basics = basics[keepers1]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
42384,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013,120,"Drama,History"
61115,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,70,Drama
67667,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
86799,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...
9939090,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,74,Drama
9939485,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,97,"Comedy,Drama,Fantasy"
9939625,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,51,Drama
9939634,tt9916190,movie,Safeguard,Safeguard,0,2020,95,"Action,Adventure,Thriller"


#### Filtering for ratings with akas dataframe for US only

In [37]:
keepers2 =ratings['tconst'].isin(akas['titleId'])
keepers2

0           True
1           True
2          False
3          False
4           True
           ...  
1321283    False
1321284    False
1321285    False
1321286    False
1321287    False
Name: tconst, Length: 1321288, dtype: bool

In [38]:
ratings = ratings[keepers2]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
4,tt0000005,6.2,2625
5,tt0000006,5.1,182
6,tt0000007,5.4,821
...,...,...,...
1321249,tt9916200,8.1,229
1321250,tt9916204,8.2,263
1321257,tt9916348,8.3,18
1321258,tt9916362,6.4,5384


### Saving modified dataframes to file for later use

In [39]:
basics.to_csv("Data/title_basics1.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas1.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings1.csv.gz",compression='gzip',index=False)